In [68]:
import cv2 # OpenCV

import numpy as np
import pandas as pd
import math

# Deep Learning
import tensorflow
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt
# Other libs
import glob
import os, sys
import shutil

In [ ]:
# Build the data generator
def bring_data_from_directory():
    datagen = ImageDataGenerator(rescale=1. / 255)
    train_generator = datagen.flow_from_directory(
            "./",
            target_size=(224, 224),
            batch_size=1,
            class_mode='categorical',
            classes=["Original", "Deepfakes"],
            shuffle=False)
    return train_generator

In [ ]:
train_generator = bring_data_from_directory()

In [202]:
# Build Convolutional Neural Network model (VGG16, pretrained on ImageNet)
def load_CNN_model():
    CNN_model = VGG16(weights='imagenet', 
                      include_top=False, 
                      input_shape=(224, 224, 3))
    
    # Retrained weights
    #weights_path = "/content/drive/My Drive/Hackathon for Good/Retrained CNN/retrainedCNN.h5"
    #CNN_model.load_weights(weights_path)
    print("VGG16 CNN model loaded.")
    print(CNN_model.summary())
    return CNN_model

58892288/58889256 [==============================] - 11s 0us/step
VGG16 CNN model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________

In [ ]:
# Build feature extraction model
CNN_model = load_CNN_model()

In [ ]:
N_VIDEOS = 1000
x_generator = None
y_lable = None
batch = 0
TIMESTEPS = 10

for x,y in train_generator:
    if batch == int(N_VIDEOS*TIMESTEPS):
        break

    print("predict on batch:", batch)
    batch+=1
    if x_generator is None:
        x_generator = CNN_model.predict_on_batch(x)
        y_label = y
    else:
        x_generator = np.append(x_generator, CNN_model.predict_on_batch(x), axis=0)
        y_label = np.append(y_label, y, axis=0)

#np.save(open('/content/drive/My Drive/Hackathon for Good/VGG_features/video_x_train_VGG16_original.npy', 'wb'), x_generator)
#np.save(open('/content/drive/My Drive/Hackathon for Good/VGG_features/video_y_train_VGG16_original.npy','wb'), y_label)

In [208]:
# Load input videos
TIMESTEPS = 10
train_fake = np.load(open('video_x_train_VGG16_deepfake.npy','rb'))
train_fake_labels = np.load(open('video_y_train_VGG16_deepfake.npy','rb'))
train_original = np.load(open('video_x_train_VGG16_original.npy','rb'))
train_original_labels = train_fake_labels.copy()
train_original_labels[:,[0, 1]] = train_original_labels[:,[1, 0]]
train_original_labels = train_original_labels[0:train_original.shape[0], :]

train_data = np.concatenate((train_fake, train_original), axis=0)
train_labels = np.concatenate((train_fake_labels, train_original_labels), axis=0)

N_CLASSES = 2
train_data = train_data.reshape(int(train_data.shape[0]/TIMESTEPS), 
                                TIMESTEPS,
                                train_data.shape[1] * train_data.shape[2] * train_data.shape[3])
train_labels = train_labels.reshape(int(train_labels.shape[0]/TIMESTEPS) , TIMESTEPS, N_CLASSES)[:, 0]

train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size=0.3, random_state=42)

In [209]:
# Build Recurrent Neural Network model (Long Short-Term Memory network)
import tensorflow
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt

# Other libs
import glob
import os, sys
from tensorflow.keras.layers import TimeDistributed, Flatten

batch_size = 300
def train_lstm_model(train_data, train_labels, H_DIM=128):
    ''' 
    Used to train the recurrent neural network model, using RMSprop optimizer

    '''
    model = Sequential()
    model.add(LSTM(H_DIM,
                   dropout=0.1,
                   input_shape=(train_data.shape[1], train_data.shape[2])))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))
    
    optimizer = RMSprop()
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    callbacks = [EarlyStopping(monitor='val_loss', 
                             patience=10, 
                             verbose=0), 
               ModelCheckpoint('video_1_LSTM_1_1024.h5', 
                               monitor='val_loss', 
                               save_best_only=True, 
                               verbose=0) ]
    
    nb_epoch = 100
    model.fit(train_data, 
              train_labels,
              validation_split=0.2,
              batch_size=batch_size,
              epochs=nb_epoch,
              callbacks=callbacks,
              shuffle=True)
  
    return model

In [210]:
# Train the recurrent neural network model
lstm_model = train_lstm_model(train_data, train_labels)

Train on 1120 samples, validate on 280 samples
Epoch 1/100
1120/1120 [==============================] - 20s 18ms/sample - loss: 1.3214 - acc: 0.4884 - val_loss: 0.7950 - val_acc: 0.4429
Epoch 2/100
1120/1120 [==============================] - 8s 7ms/sample - loss: 0.7159 - acc: 0.4973 - val_loss: 0.7296 - val_acc: 0.4429
Epoch 3/100
1120/1120 [==============================] - 8s 7ms/sample - loss: 0.7122 - acc: 0.5107 - val_loss: 0.7262 - val_acc: 0.4429
Epoch 4/100
1120/1120 [==============================] - 8s 7ms/sample - loss: 0.7156 - acc: 0.4991 - val_loss: 0.7465 - val_acc: 0.4429
Epoch 5/100
1120/1120 [==============================] - 8s 7ms/sample - loss: 0.6984 - acc: 0.5232 - val_loss: 0.7132 - val_acc: 0.4429
Epoch 6/100
1120/1120 [==============================] - 8s 7ms/sample - loss: 0.7180 - acc: 0.5107 - val_loss: 0.9068 - val_acc: 0.4429
Epoch 7/100
1120/1120 [==============================] - 8s 7ms/sample - loss: 0.7558 - acc: 0.5098 - val_loss: 0.7126 - val_acc:

In [212]:
#CNN_model.save("CNN_model.h5")
#lstm_model.save("LSTM_model.h5")